In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch
import numpy as np

In [ ]:
df = pd.read_csv('/content/train_v1_updated.csv')

In [ ]:
sentiment_labels = {
    'mycket negativ': 0,
    'lite negativ': 1,
    'neutral': 2,
    'lite positiv': 3,
    'mycket positiv': 4
}

df['label'] = df['label'].map(sentiment_labels)


In [ ]:
df.dropna(subset=['label'], inplace=True)
df['label'] = df['label'].astype(int)


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
token = "hide token"

In [ ]:
import os
os.environ["HF_TOKEN"] = "hide token"

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_name = "HI-84/HI_model"
token = "hide token"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModel.from_pretrained(model_name, token=token)

print("Model loaded successfully!")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/399k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Model loaded successfully!


In [ ]:
AutoTokenizer.from_pretrained(model_name, token=os.environ["HF_TOKEN"])

BertTokenizerFast(name_or_path='HI-84/HI_model', vocab_size=50325, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
model_name = "HI-84/HI_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(sentiment_labels))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HI-84/HI_model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9940 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2485 [00:00<?, ? examples/s]

In [ ]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [ ]:
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, WeightedRandomSampler
from torch.nn import CrossEntropyLoss

class CustomTrainer(Trainer):
    def _init_(self, *args, class_weights=None, oversample_classes=None, **kwargs):
        super()._init_(*args, **kwargs)
        self.class_weights = class_weights
        self.oversample_classes = oversample_classes

    def get_train_dataloader(self):
        if self.oversample_classes:
            labels = np.array(self.train_dataset['label'])
            class_counts = np.bincount(labels)
            weights = 1. / class_counts

            if self.class_weights is not None:
                for i, weight_val in enumerate(self.class_weights):
                    weights[i] *= weight_val

            sample_weights = weights[labels]
            sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
            return DataLoader(self.train_dataset, batch_size=self.args.per_device_train_batch_size, sampler=sampler)
        else:
            return super().get_train_dataloader()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

In [ ]:
from datasets import Dataset
hf_dataset = Dataset.from_pandas(df)


In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)
tokenized_df = hf_dataset.map(tokenize, batched=True)



Map:   0%|          | 0/12425 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

tokenized_train = train_dataset.map(tokenize, batched=True)
tokenized_test = test_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/9940 [00:00<?, ? examples/s]

Map:   0%|          | 0/2485 [00:00<?, ? examples/s]

In [ ]:
!pip install -q transformers datasets

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score

tokenizer = AutoTokenizer.from_pretrained("HI-84/HI_model")

def tokenize(batch):
   return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_df = tokenized_df.map(tokenize, batched=True)
tokenized_df = tokenized_df.remove_columns(["text"])

model = AutoModelForSequenceClassification.from_pretrained("HI-84/HI_model", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}




Map:   0%|          | 0/12425 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HI-84/HI_model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train.shuffle(seed=42).select(range(2000)),
    eval_dataset=tokenized_test.select(range(500)),
    compute_metrics=compute_metrics
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("HI-84/HI_model", num_labels=5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HI-84/HI_model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df['label'].value_counts()
print("Unique labels:", df['label'].nunique())


Unique labels: 5


In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [ ]:
df.columns

Index(['text', 'label'], dtype='object')

In [ ]:
train_df = train_df.rename(columns={"sentence": "text"})
val_df = val_df.rename(columns={"sentence": "text"})


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/9940 [00:00<?, ? examples/s]

Map:   0%|          | 0/2485 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text", "__index_level_0__"])
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["text", "__index_level_0__"])


In [ ]:
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)


/tmp/ipython-input-954432133.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
train_dataset.column_names


['text', 'label', '__index_level_0__']

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maleehaashfaq0 (maleehaashfaq0-hi-ai-research) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.245500
20,1.103600
30,1.178600
40,1.146600
50,1.187300
60,1.285200
70,1.144400
80,1.049300
90,1.052000
100,0.899500


TrainOutput(global_step=1866, training_loss=0.7248511189177617, metrics={'train_runtime': 795.4001, 'train_samples_per_second': 37.491, 'train_steps_per_second': 2.346, 'total_flos': 1961545752069120.0, 'train_loss': 0.7248511189177617, 'epoch': 3.0})

In [ ]:
output_model_dir = "./swedbert_sentiment_model"
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

print(f"Model and tokenizer saved to {output_model_dir}")

# Basic ML backend placeholder (swedbert_backend.py)
# This would typically be a Flask/FastAPI app. For now, just a conceptual outline.
backend_code = """
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model_path = "./swedbert_sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval() # Set model to evaluation mode

sentiment_map = {
    0: 'mycket negativ',
    1: 'lite negativ',
    2: 'neutral',
    3: 'lite positiv',
    4: 'mycket positiv'
}
#  Add this route BEFORE app.run()
@app.route("/")
def index():
    return "Flask ML backend is running!"

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    text = data.get('text', '')

    if not text:
        return jsonify({'error': 'No text provided'}), 400

    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_class_id = torch.argmax(probabilities, dim=1).item()
        score = probabilities[0][predicted_class_id].item()

    predicted_label = sentiment_map[predicted_class_id]

    # Apply the post-processing logic for filtering
    # non-neutral rows with score >= 0.8
    # neutral rows only if score >= 0.9

    # Convert predicted_class_id back to score -2 to +2
    # 0: -2, 1: -1, 2: 0, 3: 1, 4: 2
    sentiment_score = predicted_class_id - 2

    # This filtering logic is for *after* the model delivers, during auto-labeling
    # For the prediction endpoint, we should return the raw prediction and probability
    # The filtering will be done by the client consuming this API

    return jsonify({
        'text': text,
        'predicted_label': predicted_label,
        'probability': score,
        'sentiment_score': sentiment_score
    })

if __name__ == '__main__':
    # This will run on port 9090 as specified in the agreement
    app.run(host='0.0.0.0', port=9090)
"""

with open("swedbert_backend.py", "w") as f:
    f.write(backend_code)

print("swedbert_backend.py created.")

# Instructions for Google Colab
colab_instructions = """
# Google Colab Setup and Execution Instructions

This script `train_swedbert.py` will fine-tune the SwedBERT model for sentiment analysis.

**1. Upload Files:**
   - Upload `train_v1_updated.csv` and `train_swedbert.py` to your Google Colab environment.

**2. Install Dependencies:**
   Run the following in a Colab cell to install necessary libraries:
   ```bash
   !pip install transformers datasets scikit-learn pandas torch flask
   ```

**3. Run the Training Script:**
   Run the training script from a Colab cell:
   ```bash
   !python train_swedbert.py
   ```

   This will:
   - Load and preprocess your data.
   - Fine-tune the SwedBERT model.
   - Save the trained model and tokenizer to `./swedbert_sentiment_model/`.
   - Create a placeholder Flask backend script `swedbert_backend.py`.

**4. Deploying the ML Backend (Conceptual):**
   The `swedbert_backend.py` is a basic Flask application. To deploy it for Label Studio or a public API, you would typically use a service like Ngrok (for temporary local exposure) or a cloud platform (e.g., Google Cloud Run, AWS EC2, Azure App Service) for production deployment.

   **To run the backend locally in Colab (for testing purposes, requires port forwarding/tunneling like ngrok):**
   ```bash
   !python swedbert_backend.py
   ```
   *Note: Directly exposing ports from Colab requires additional tools like `ngrok` or `localtunnel` for external access, which is beyond the scope of this script but essential for Label Studio to connect.*

**5. Retraining:**
   To retrain with new data, simply replace `train_v1_updated.csv` with your new `gold-label CSV` and re-run `!python train_swedbert.py`.

**Important Considerations:**
- **GPU Usage:** Ensure you are running your Colab notebook with a GPU runtime (`Runtime -> Change runtime type -> GPU`).
- **Data Volume:** For very large datasets, consider using `Trainer`'s `fp16=True` argument in `TrainingArguments` for mixed-precision training to save memory.
- **Hyperparameters:** The `num_train_epochs`, `per_device_train_batch_size`, etc., in `TrainingArguments` can be tuned for better performance.
- **Oversampling/Class Weights:** The `CustomTrainer` implements the specified logic. Monitor training logs to ensure it's having the desired effect on class balance.
"""

with open("colab_instructions.md", "w") as f:
    f.write(colab_instructions)

print("colab_instructions.md created.")

Model and tokenizer saved to ./swedbert_sentiment_model
swedbert_backend.py created.
colab_instructions.md created.


In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime
SOCIAL_MEDIA_API_ENDPOINT = "https://api.example.com/social_media_feed"
API_KEY = "YOUR_SOCIAL_MEDIA_API_KEY"
def fetch_social_media_data(query, num_posts=100):
    """
    Fetches social media posts based on a query.
    In a real application, this would interact with a social media API.
    """
    print(f"Fetching {num_posts} posts for query: ", query)
    mock_data = []
    for i in range(num_posts):
        mock_data.append({
            "id": f"post_{datetime.now().strftime('%Y%m%d%H%M%S%f')}_{i}",
            "text": f"This is a sample post about {query}. Sentiment is neutral for now. Post number {i}.",
            "timestamp": datetime.now().isoformat()
        })
    return mock_data

def preprocess_post(post_text):
    processed_text = post_text.replace("http://", "").replace("https://", "")
    processed_text = ' '.join(word for word in processed_text.split() if not word.startswith(('@', '#')))
    return processed_text.strip()
if __name__ == "__main__":
    stocks_to_monitor = ["Microsoft", "Ericsson", "Volvo"]
    all_processed_data = []

    for stock in stocks_to_monitor:
        raw_posts = fetch_social_media_data(stock, num_posts=50)
        for post in raw_posts:
            processed_text = preprocess_post(post["text"])
            all_processed_data.append({
                "stock": stock,
                "original_text": post["text"],
                "processed_text": processed_text,
                "timestamp": post["timestamp"]
            })

    df_collected = pd.DataFrame(all_processed_data)
    output_filename = "collected_social_media_data.csv"
    df_collected.to_csv(output_filename, index=False)
    print(f"Collected and preprocessed data saved to {output_filename}")

    print("\n--- Demonstrating sending data to ML Backend (conceptual) ---")
    if not df_collected.empty:
        sample_post = df_collected.iloc[0]
        ml_backend_url = "http://localhost:9090/predict"

        try:
            print(f"Sending post for prediction: {sample_post['processed_text']}")
            response = requests.post(ml_backend_url, json={'text': sample_post['processed_text']})
            response.raise_for_status()
            prediction_result = response.json()
            print("Prediction Result:", prediction_result)
        except requests.exceptions.ConnectionError:
            print(f"Error: Could not connect to ML backend at {ml_backend_url}. Is it running?")
        except requests.exceptions.HTTPError as e:
            print(f"HTTP Error: {e.response.status_code} - {e.response.text}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
    else:
        print("No data collected to send for prediction.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Fetching 50 posts for query:  Microsoft
Fetching 50 posts for query:  Ericsson
Fetching 50 posts for query:  Volvo
Collected and preprocessed data saved to collected_social_media_data.csv

--- Demonstrating sending data to ML Backend (conceptual) ---
Sending post for prediction: This is a sample post about Microsoft. Sentiment is neutral for now. Post number 0.


INFO:werkzeug:127.0.0.1 - - [11/Aug/2025 07:34:14] "POST /predict HTTP/1.1" 200 -


Prediction Result: {'predicted_label': 'neutral', 'probability': 0.9964225888252258, 'sentiment_score': 0, 'text': 'This is a sample post about Microsoft. Sentiment is neutral for now. Post number 0.'}


In [ ]:
!ngrok config add-authtoken hide token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask-ngrok pyngrok transformers torch --quiet

from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from pyngrok import ngrok
import threading
app = Flask(__name__)

model_path = "./swedbert_sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

sentiment_map = {
    0: 'mycket negativ',
    1: 'lite negativ',
    2: 'neutral',
    3: 'lite positiv',
    4: 'mycket positiv'
}
@app.route("/")
def index():
    return " Flask ML backend is running!"
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    text = data.get('text', '')

    if not text:
        return jsonify({'error': 'No text provided'}), 400

    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_class_id = torch.argmax(probabilities, dim=1).item()
        score = probabilities[0][predicted_class_id].item()

    predicted_label = sentiment_map[predicted_class_id]
    sentiment_score = predicted_class_id - 2

    return jsonify({
        'text': text,
        'predicted_label': predicted_label,
        'probability': score,
        'sentiment_score': sentiment_score
    })

def run_flask():
    app.run(port=9090)

thread = threading.Thread(target=run_flask)
thread.start()


public_url = ngrok.connect(9090)
print(f" Public URL for prediction: {public_url}/predict")


 * Serving Flask app '__main__'
 * Debug mode: off
 Public URL for prediction: NgrokTunnel: "https://b02651a211b0.ngrok-free.app" -> "http://localhost:9090"/predict


Address already in use
Port 9090 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
model.save_pretrained("swedbert_sentiment_model")
tokenizer.save_pretrained("swedbert_sentiment_model")


('swedbert_sentiment_model/tokenizer_config.json',
 'swedbert_sentiment_model/special_tokens_map.json',
 'swedbert_sentiment_model/vocab.txt',
 'swedbert_sentiment_model/added_tokens.json',
 'swedbert_sentiment_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("swedbert_sentiment_model")
tokenizer = AutoTokenizer.from_pretrained("swedbert_sentiment_model")
inputs = tokenizer("Jag tycker det är väldigt bra.", return_tensors="pt")
outputs = model(**inputs)
print(outputs.logits.argmax().item())


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


3


In [ ]:
!pip install pyngrok


In [ ]:
public_url = ngrok.connect(9090)
print("Public URL:", public_url)


In [ ]:
!python swedbert_backend.py


In [ ]:
!python train_swedbert.py


In [ ]:
!pip install numpy==1.26.4


In [ ]:
model_path = "SwedBERT-sentiment-v1"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

print("Model saved to:", model_path)


In [ ]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

app = FastAPI()
model = AutoModelForSequenceClassification.from_pretrained("maleeha123/SwedBERT-sentiment-v1")
tokenizer = AutoTokenizer.from_pretrained("maleeha123/SwedBERT-sentiment-v1")

In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="maleeha123/SwedBERT-sentiment-v1",
    folder_path="SwedBERT-sentiment-v1",
    path_in_repo=".",
    token="hide token"
)

In [ ]:
!pip install pyngrok

In [ ]:
!python swedbert_backend.py

2025-08-07 08:19:49.515706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754554789.537506   20384 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754554789.543658   20384 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754554789.558991   20384 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754554789.559024   20384 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754554789.559027   20384 computation_placer.cc:177] computation placer alr